# 10. CODE COMPLETION

1. [x] введение
2. [x] grammformer
3. [x] упражнение
4. [x] ссылки

# 1. Введение

Автодополнение --- функция в программах, предусматривающих интерактивный ввод текста (редакторы, оболочки командной строки, браузеры и т. д.), по дополнению текста по введённой его части.
![autocompletion](https://upload.wikimedia.org/wikipedia/commons/c/cd/Autoaanvullen.PNG)

Можно рассматривать как вариант задачи language modelling: продолжить код.
Однако на практике бывает сложно завершить код, например, из-за того, что трудно предсказать строковые литералы.
В этом случае модель начинает "галлюцинировать" --- генерировать правдоподобный, но неправильный код.
При этом, генерация проходит в авторегрессионном режиме, то есть последующие части зависят от предыдущих, даже если они неправильные.

Можно ли решить эту проблему?

# 2. Grammformer

В качестве решения можно генерировать код с "дырами", который модель вставляет, если не уверена в ответе.
Подход описан в работе [Learning to Complete Code with Sketches](https://arxiv.org/abs/2106.10158).

![](./res/10_grammformer_paper.png)

## 2.1 Идея

В случае, если модель не уверена, будем вставлять специальный токен ⬛.
Пример работы предложенного подхода и других методов:

![](./res/10_grammformer_example.png)


Что необходимо обеспечить, чтобы появились ⬛?
Поскольку мы рассматриваем авторегрессионные модели, то в случае появления токена ⬛, этот токен на следующим шаге попадёт на вход модели. Следовательно, модель должна уметь его корректно обрабатывать, то есть она должна быть обучена на данных, содержащих ⬛.

## 2.2 Метрики

*Скетч* --- код с дырами (⬛).
> Например, `t = foo(⬛)`.

Какой скетч будем считать хорошим? Два условия:
1. тот, который может быть приведён (заменяя ⬛ на части кода) к правильному ответу
2. точен, насколько это возможно

### $RegexAcc$

Какую метрику будем использовать для этого? Сначала введём обозначения:
- $x$ --- код на входе
- $y^*$ --- правильный ответ, код без токена ⬛)
- $\hat{y}$ --- выход модели, скетч (может содержать токена ⬛)
- $toRegex(\hat{y})$ --- функция, которая превращает скетч $\hat{y}$ в регулярное выражения, заменяя токен ⬛ на `.+` (любое выражение)
- $matches(r, s)$ --- проверка того, что строка $s$ удовлетворяет регулярному выражению $r$ ($1$, иначе $0$)
- $nTokens(\hat{y})$ --- количество символов в скетче $\hat{y}$, отличных от ⬛

Тогда

$$RegexAcc(\hat{y}, y^*) = matches(toRegex(\hat{y}), y^*) \frac{nTokens(\hat{y})}{nTokens(y^*)}$$.

Первый множитель $RegexAcc$ оценивает условие 1, второй множитель --- условие 2.


### $ROUGE$

$ROUGE$ --- метрика из NLP, использующая в задаче суммаризации текста. В основе лежит сравнение на $n$-граммах.
Идея: скетч можно рассматривать как суммаризацию исходного кода, поэтому хотим использовать $Rouge$.

Пусть
- $EraseHoles(\hat{y})$ --- функция, которая удаляет все символы ⬛ из $\hat{y}$.

Тогда используем
$$Rouge_{F1}(EraseHoles(\hat{y}), y^*).$$
Эта метрика менее строгая к ошибкам и поощряет правдоподобные скетчи.

## 2.3 Линейная генерация скетчей

1. используем стандартную генеративную языковую модель
2. дополняем словарь токеном ⬛
3. используем RL подход
4. используем функцию награды
$$r(\hat{y}, y^*) = \frac{1}{2}(RegexAcc(\hat{y}, y^*) + ROUGE_{F1}(EraseHoles(\hat{y}), y^*))$$
для оценки качества выхода и вычисления функции потерь
5. используем [self-critical policy gradient training](https://arxiv.org/abs/1612.00563)

Для выхода модели $\hat{y}$ минимизируем

$$L(x, y^*) = (r(\hat{y}, y^*) − \tilde{r}(x))⋅L_{gen}(x, \hat{y}),$$

где $\tilde{r}(x)$ --- награда, полученная благодаря прогнозу на версии модели, которая на данный момент достигла наилучшего результата, а $L_{gen}$ --- это функция потерь генеративной модели. Таким образом, модель вознаграждается, если улучшает предыдущую наилучшую политику (policy) в отношении $r$.

Используем полный Трансформер для "перевода" $x$ в $y$ (режим динамического декодирования).

## 2.4 Генерация скетчей, основанная на грамматике

Предыдущий подход работал не очень хорошо: токен ⬛ вставал не всегда в ожидаемое место.
> Например, в случае `szconv.⬛)` не хватает левой скобки.

Таким образом, авторы пришли к генерации скетчей, основанной на грамматике.

Обозначения:

- $\Sigma$ --- множество терминальных символов
- $N$ --- множество нетерминальных символов
- $s \in N$ --- корень
- $R$ --- множество правил
- $\langle \Sigma, N, S, R \rangle$ --- контекстно-свободная грамматика
- через $\langle NonTerminalName \rangle$ будем обозначать нетерминальные вершины

Grammformer --- seq2seq-модель:
- на входе последовательность $x = x_0x_1 \ldots x_n$
- нетерминальная вершина $x_i$ заменяется на новую последовательность символов в соответствии с правилами грамматики

> Пример генерации скетча `r = x * (⬛ - foo(args))`. Синий цвет --- терминальные вершины.
> ![](https://www.miltos.allamanis.com/publicationfiles/guo2021learning/guo2021learning.png)


Grammformer работает в два шага:
1. модель $P_s$ выбирает нетерминальную вершину в $x$ для раскрытия
2. модель $P_e$ раскрывает выбранную вершину

Пусть $N(x) = \{i ~|~ x_i\in N\} \cup \{🚫\}$ --- индексы нетерминальных символов в последовательности $x$ и специальный символ окончания раскрытия.
Для заданного $x$ модель $P_s$ рассчитывает вероятностное распределение на $N(x)$.
В свою очередь, для заданного $x$ и индекса $i \in N(x)$ модель $P_e$ рассчитывает распределение вероятностей на  последотельностях раскрытия $u \in (\Sigma \cup N)^*$.

![algorithm](./res/10_grammformer_algorithm.png)

Функция $NonTerminalsToHoles(\cdot)$ заменяет все оставшиеся нетерминальные символы на ⬛.


#### Модели $P_s$ и $P_e$

Используется один энкодер из Трансформера для обеих моделей.
Он преобразует последовательность $x = x_0 \ldots x_n$ в вектора $e_0 \ldots e_n$ из $\mathrm{R}^D$ ($D$ --- гиперпараметр).

Модель $P_s$:

$$P_s(i|x) = softmax_{i \in N(x)}(f(e_i)),$$
где $f$ --- обучаемая FFN-сеть ($e_🚫$ --- представление $CLS$-токена).

Модель $P_e$:

$$P_e (u ∣ x, i) = \prod_{j=1}^{m} P_{dec} (u_j ∣ e_0, \ldots, e_n, i, u_{<j})$$

--- [relational](https://arxiv.org/abs/1911.04942) авторегрессионый декодер из Трансформера (добавлены дополнительные связи в механизме аттеншна: дополнительная связь каждого токена с раскрываемым нетерминальным токеном $x_i$).


#### Функция потерь

$$L(x, y^*) = (r(\hat{y}, y^*) − \tilde{r}(x))⋅ \sum_{t=0}^{T}(− \log P_s (i^{(t)} | x^{(t)}) - I(i^{(t)} \neq 🚫) \cdot \log P_e((u^{(t)}_{⊚i^{(t)}})^* | x^{(t)}, i^{(t)})),$$


где $\tilde{r}(x)$ – это награда, полученная за версии моделей $P_s$ и $P_e$, набравшие на данный момент лучший результат.
Остальная часть цели соответствует итерациям цикла в алгоритме, где
- $t$ --- индекс итерации,
- $\hat{y}$ --- прогнозируемый скетч,
- $y^*$ --- истинная последовательность терминальных символов,
- $I(\cdot)$ --- индикаторная функция.

## 2.3 Процесс генерации

![example](./res/10_grammformer_generation.png)

## 2.4 Сравнение

- $LM$ --- языковая модель (на основе декодера)
- $L \rightarrow R$ --- полный трансформер
- $LM + 🚫$ --- модель обучена останавливать генерацию за счёт добавления токена ⬛
- $L \rightarrow R + 🚫$ --- модель обучена останавливать генерацию за счёт добавления токена ⬛

Эти модели могут генерировать только скетчи, которые являются префиксами целевого кода, т.е. это соответствует стандартной генеративной модели на уровне токенов с обучаемой способностью остановки.

### Качество

![](./res/10_grammformer_perfomance.png)

### Длина ответа

![](./res/10_grammformer_length.png)


## 3. Упражнение

Провести исследование продуктов по автодополнению кода:
- какие продукты есть
- какие компании
- когда
- где используются
- сравнение

# 4. Ссылки

- https://gousios.org/courses/ml4se/autocomplete.html
- https://youtu.be/cF8OAS4_w3k
- https://software-lab.org/teaching/summer2020/asdl/slides_02_rnn-based_02_completion.pdf
- https://dl.acm.org/doi/pdf/10.1145/2594291.2594321
- https://github.com/saltudelft/ml4se#code-completion
- Guo et al - Learning to complete code with sketches 2022
- Liu et al - Multi-task learning based pre-trained language model for code completion 2020
- Lu et al - ReACC A retrieval-augmented code completion framework 2022